In [1]:
import pandas as pd
import numpy as np
import math
import random
import nltk
nltk.data.path.append('.')

In [2]:
#Load the data
with open("en_US.twitter.txt", "r", encoding="utf-8") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


# Pre-process the data

Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
2. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
3. Assign sentences into train or test sets.
4. Find tokens that appear at least N times in the training data.
5. Replace tokens that appear less than N times by <unk>

In [4]:
def split_to_sentences(data) :
    sentences = data.split("\n")
    sentences = [s.strip() for s in sentences] #- Remove leading and trailing spaces from each sentence
                                               # - Drop sentences if they are empty strings.
    sentences = [s for s in sentences if len(s)>0] #remove null sentences
    return sentences

In [5]:
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_to_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

In [6]:
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences :
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences


In [7]:
# test your code
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [8]:
#def get_tokenized_data
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [9]:
# test your function
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

In [10]:
# train test split
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)
train_size = int(len(tokenized_data)*0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [11]:
print(f"{len(tokenized_data)} data are split into {len(train_data)} train and {len(test_data)} test set")

print("First training sample:")
print(train_data[0])
      
print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


In [12]:
# count_words
def count_words(tokenized_sentences):
    count_words = {}
    for sentence in tokenized_sentences :
        for token in sentence :
            if token not in count_words.keys() :
                count_words[token] = 1
            else :
                count_words[token] = count_words[token] + 1
    return count_words

In [13]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

In [14]:
# create closed vocabulary this vocabulary contains those words which have frequence greater than and equal to certain thresold
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items():
        if cnt >= count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [15]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['.', 'are']


# The words that appear 'count_threshold' times or more are in the 'closed vocabulary.

All other words are regarded as 'unknown'.

Replace words not in the closed vocabulary with the token "<unk>".

In [17]:
#
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences :
        replaced_sentence = []
        for token in sentence :
            if token in vocabulary :
                replaced_sentence.append(token)
            else :
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence) 
    return replaced_tokenized_sentences

In [18]:
#test above function
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


# Now we are ready to process our data by combining the functions that you just implemented.

1. Find tokens that appear at least count_threshold times in the training data.

2. Replace tokens that appear less than count_threshold times by "<unk>" both for training and test data.

In [20]:
def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary, unknown_token="<unk>")
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary, unknown_token="<unk>")
    return train_data_replaced ,  test_data_replaced , vocabulary
    

In [21]:
# test above function code
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train, 
                                                           tmp_test, 
                                                           count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In [22]:
#Pre-process the train and test data
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, 
                                                                        test_data, 
                                                                        minimum_freq)

In [23]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14823


# Develop n-gram based language models

In [25]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}
    for sentence in data:
        sentence = [start_token] * n+ sentence + [end_token]
        sentence = tuple(sentence)
        m = len(sentence) if n==1 else len(sentence)-(n-1)
        for i in range(m): 
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys():
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    return n_grams

In [26]:
# test your code
# CODE REVIEW COMMENT: Outcome does not match expected outcome
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 4))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>', '<s>', '<s>'): 2, ('<s>', '<s>', '<s>', 'i'): 1, ('<s>', '<s>', 'i', 'like'): 1, ('<s>', 'i', 'like', 'a'): 1, ('i', 'like', 'a', 'cat'): 1, ('like', 'a', 'cat', '<e>'): 2, ('<s>', '<s>', '<s>', 'this'): 1, ('<s>', '<s>', 'this', 'dog'): 1, ('<s>', 'this', 'dog', 'is'): 1, ('this', 'dog', 'is', 'like'): 1, ('dog', 'is', 'like', 'a'): 1, ('is', 'like', 'a', 'cat'): 1}


In [27]:
def estimate_probability(word, previous_n_gram, 
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram) #previous n-number of words of predicted word
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts  else 0
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,) #previous part of predicted part and predicted word
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    return probability

In [28]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)

bigram_counts = count_n_grams(sentences, 2)

tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


# Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.

In [30]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
     # convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)
    # add <e> <unk> to the vocabulary
    # <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    return probabilities

In [31]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'dog': 0.09090909090909091,
 'is': 0.09090909090909091,
 'i': 0.09090909090909091,
 'a': 0.09090909090909091,
 'like': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'this': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [32]:
# Example DataFrame
count_matrix = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Row sums
row_sums = count_matrix.sum(axis=1)

# Element-wise division
result = count_matrix.div(row_sums, axis=0)
print(result)


          A         B         C
0  0.083333  0.333333  0.583333
1  0.133333  0.333333  0.533333
2  0.166667  0.333333  0.500000


# Count and probability matrices

As we have seen so far, the n-gram counts computed above are sufficient for computing the probabilities of the next word.

1. It can be more intuitive to present them as count or probability matrices.
    
2. The functions defined in the next cells return count or probability matrices.
    
3. This function is provided for you.

In [34]:
import pandas as pd
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys() :
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))   

    #mapping from n_gram to row number
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    #mapping from word to column number
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items() :
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary :
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i,j] = count
    count_matrix = pd.DataFrame(count_matrix, index = n_grams, columns = vocabulary)
    return count_matrix

In [35]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,dog,is,i,a,like,cat,this,<e>,<unk>
"(is,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(dog,)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like,)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>,)",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
"(i,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(this,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Show trigram counts
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))


trigram counts


,dog,is,i,a,like,cat,this,<e>,<unk>
"(<s>, this)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(is, like)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(this, dog)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(dog, is)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(<s>, <s>)",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [38]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,dog,is,i,a,like,cat,this,<e>,<unk>
"(is,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(dog,)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(like,)",0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>,)",0.090909,0.090909,0.181818,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909
"(i,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909
"(this,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [39]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

trigram probabilities


,dog,is,i,a,like,cat,this,<e>,<unk>
"(<s>, this)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(i, like)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(like, a)",0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909
"(is, like)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a, cat)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(this, dog)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(dog, is)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(<s>, i)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(<s>, <s>)",0.090909,0.090909,0.181818,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909


# Perplexity

In [97]:
#calculate perplexity
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = 1.0):
    n = len(list(n_gram_counts.keys())[0])
    sentence = ['<s>']*n + sentence + ['<e>']
    sentence = tuple(sentence)
    N = len(sentence)
    product_prob = 1
    for t in range(n, N): 
        n_gram_test = sentence[t-n:t]
        word = sentence[t]
        prob = estimate_probability(word, n_gram_test, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
        product_prob = product_prob * (1/prob)
    perflexity = product_prob**(1/float(N))
    return perflexity

In [99]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


In [117]:
#prdiction next word
def pridcted_next_word(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0) :
    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0)
    prev_prob = 0
    for word, prob in probabilities.items() :
        if prev_prob < prob :
            prev_prob = prob
            prev_word = word
    return prev_word    
        
    


In [130]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
pridcted_next_word("a", unigram_counts, bigram_counts, unique_words, k=1)

'cat'

# Build an auto-complete system

In [134]:
# suggest_a_word
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    n = len(list(n_gram_counts.keys())[0])
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0)
    max_prob = 0
    suggestion = None
    for word, prob in probabilities.items() :
        if start_with != None: 
            if not word.startswith(start_with):
                continue 
        if max_prob < prob :
            max_prob = prob
            suggestion = word
    return suggestion, max_prob    

In [140]:
# For testing above code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'd'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `d`
	and the suggested word is `dog` with a probability of 0.0909


# Multiple suggesestion

In [163]:
# Multiple suggestion
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    num_models = len(n_gram_counts_list)
    suggestions = []
    for i in range(num_models-1) :
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        suggestion = suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k = k)
        suggestions.append(suggestion)
    return suggestions    

In [165]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('dog', 0.1111111111111111),
 ('dog', 0.1111111111111111)]

# Suggest multiple words using n-grams of varying length

In [168]:
len(train_data_processed)

38368

In [182]:
# generate multiple size n-grams
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


# Some Example of our model

In [184]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.02766367370678687),
 ('have', 0.00013485267345425124),
 ('have', 0.00013488905375328792),
 ('i', 6.745362563237774e-05)]

In [186]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest5 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)

The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[('to', 0.014050206069689023),
 ('to', 0.004697320542507443),
 ('to', 0.0009423167530457024),
 ('to', 0.00040439441935701285)]

In [188]:
previous_tokens = ["hey", "how", "are"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)


The previous words are ['hey', 'how', 'are'], the suggestions are:


[('you', 0.023424142254644932),
 ('you', 0.0035589578297072254),
 ('you', 0.00013489815189531904),
 ('i', 6.745362563237774e-05)]

In [190]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest7 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest7)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.0239720461563465),
 ('?', 0.002888086642599278),
 ('?', 0.001613228473482557),
 ('<e>', 0.00013489815189531904)]

In [192]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.0239720461563465),
 ('?', 0.002888086642599278),
 ('?', 0.001613228473482557),
 ('<e>', 0.00013489815189531904)]